# File Ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()
from adamacs.pipeline import subject, behavior, session, event, trial, scan
from adamacs.ingest import behavior as ibe


[2023-07-24 17:09:00,230][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-07-24 17:09:00,276][INFO]: Connected tobiasr@172.26.128.53:3306


Manual entry

### BPod Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [2]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir


In [3]:

# bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FbB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
# bpod_path = "TR_WEZ-8701_2023-01-13_scan9FG1THFD_sess9FG1TAXY/scan9FG1THFD_WEZ-8701_StimArenaMaster_20230113_172307.mat"

# root_dirs = dj.config["custom"]["exp_root_data_dir"]
# bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

scansi = "scan9FJ842C3"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
bpod_path_full = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*mat"))[0]


print(f"Full: {bpod_path_full}")

Full: /datajoint-data/data/tobiasr/RN_OPI-1681_2023-07-24_scan9FJ842C3_sess9FJ842C3/scan9FJ842C3_OPI-1681_StimArenaAllPorts_20230724_163510.mat


In [4]:
bpod_object = Bpodfile(bpod_path_full)

In [5]:
bpod_object.ingest('sess9FJ842C3', 'scan9FJ842C3')

/datajoint-data/data/tobiasr/RN_OPI-1681_2023-07-24_scan9FJ842C3_sess9FJ842C3/scan9FJ842C3_RN_OPI-1681_2032.h5


/home/tobiasr/miniconda3/envs/datajoint/lib/python3.8/site-packages/pywavesurfer/ws.py:46: RuntimeWarning: You are reading a WaveSurfer file version this module was not tested with: file version 1.0.6, latest version tested: 0.982
  warnings.warn('You are reading a WaveSurfer file version this module was not tested with: '


BPod items to be inserted:
	Subject : OPI-1681
	Time    : 2023-07-24 16:37:28
	N Trials: 6
	N Events: 39


In [6]:
bpod_object.trials()

TypeError: 'dict' object is not callable

### Initial check of tables

In [ ]:
# from adamacs.pipeline import session, event, trial

# session.Session.delete()


# print('Sessions:', len(session.Session()))
# print('Trials  :', len(trial.Trial()))
# print('Events  :', len(event.Event()))

In [ ]:
scan_key

In [7]:
(event.BehaviorRecording() & scan_key)

session_id,scan_id,recording_start_time,recording_duration,recording_notes
sess9FJ842C3,scan9FJ842C3,2023-07-24 16:37:16,280.0,


In [ ]:
dj.Diagram(behavior)+dj.Diagram(event)+dj.Diagram(trial)

## Automated BPod ingestion

The function is designed ask for a confirmation before entered into the schema.

In [ ]:
session.Session()

In [ ]:
bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
# bpod_path = "TR_WEZ-8701_2023-01-13_scan9FG1THFD_sess9FG1TAXY/scan9FG1THFD_WEZ-8701_StimArenaMaster_20230113_172307.mat"

root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

In [ ]:
bpod_path

In [ ]:
bpod_object = Bpodfile(bpod_path_full)


In [ ]:
bpod_object.ingest('sess9FB2LN5C', 'scan9FB2LN5C')

In [ ]:
event.Event()

In [ ]:
scansi = "scan9FJ842C3"
set((event.Event & f'scan_id = "{scansi}"').fetch('event_type'))

In [8]:
(trial.TrialEvent & f'scan_id = "{scansi}"' & "trial_id = 0").fetch(format = "frame", order_by = "event_start_time")

Empty DataFrame
Columns: []
Index: [(sess9FJ842C3, scan9FJ842C3, 0, bpod_at_port, 12.2132), (sess9FJ842C3, scan9FJ842C3, 0, reward, 12.2232), (sess9FJ842C3, scan9FJ842C3, 0, bpod_reward, 12.3132), (sess9FJ842C3, scan9FJ842C3, 0, bpod_cue, 126.641), (sess9FJ842C3, scan9FJ842C3, 0, bpod_at_target, 126.651), (sess9FJ842C3, scan9FJ842C3, 0, bpod_in_port_4, 129.453), (sess9FJ842C3, scan9FJ842C3, 0, bpod_drinking, 129.609)]

Check that insertion worked:

In [10]:
trial.TrialEvent & 'trial_id=0' & scan_key

session_id,scan_id,trial_id trial number (1-based indexing),event_type,event_start_time (second) relative to recording start
sess9FJ842C3,scan9FJ842C3,0,bpod_at_port,12.2132
sess9FJ842C3,scan9FJ842C3,0,bpod_at_target,126.651
sess9FJ842C3,scan9FJ842C3,0,bpod_cue,126.641
sess9FJ842C3,scan9FJ842C3,0,bpod_drinking,129.609
sess9FJ842C3,scan9FJ842C3,0,bpod_in_port_4,129.453
sess9FJ842C3,scan9FJ842C3,0,bpod_reward,12.3132
sess9FJ842C3,scan9FJ842C3,0,reward,12.2232


In [11]:
event.Event & scan_key

session_id,scan_id,event_type,event_start_time (second) relative to recording start,event_end_time (second) relative to recording start
sess9FJ842C3,scan9FJ842C3,aux_bpod_reward,13.7745,13.8745
sess9FJ842C3,scan9FJ842C3,aux_bpod_reward,24.231,24.331
sess9FJ842C3,scan9FJ842C3,aux_bpod_reward,37.0155,37.1155
sess9FJ842C3,scan9FJ842C3,aux_bpod_reward,64.1573,64.2573
sess9FJ842C3,scan9FJ842C3,aux_bpod_reward,213.386,213.486
sess9FJ842C3,scan9FJ842C3,aux_bpod_tone,10.9724,11.2724
sess9FJ842C3,scan9FJ842C3,aux_bpod_tone,13.7747,14.0747
sess9FJ842C3,scan9FJ842C3,aux_bpod_tone,19.0091,19.3092
sess9FJ842C3,scan9FJ842C3,aux_bpod_tone,24.2312,24.5312
sess9FJ842C3,scan9FJ842C3,aux_bpod_tone,30.8997,31.1996


We can also interact with bpod objects. For example:

In [34]:
bpod_object.trial(0).events

{'bpod_cue': 117.23,
 'bpod_at_target': 117.24,
 'bpod_at_port': 2.8023000000000025,
 'bpod_reward': 2.9023000000000025,
 'bpod_in_port_4': 120.0423,
 'bpod_drinking': 120.1977}

In [33]:
bpod_object.trial(0).attributes

{'error': False, 'timeout': False}

In [66]:
bpod_object.trial_data[1]['Events']

{'SoftCode10': 3.4556999999999998,
 'Tup': array([3.4657, 8.7877, 8.8685, 8.8686, 8.9686]),
 'HiFi1_1': array([3.4658, 8.6878]),
 'PA1_Port1In': 8.687700000000001,
 'PA1_Port1Out': 8.9627}

In [67]:
bpod_object.trial_data[1]['States']

{'WaitForPosTriggerSoftCode': array([0.    , 3.4557]),
 'CueDelay': array([3.4557, 3.4657]),
 'WaitForResponse': array([3.4657, 8.6877]),
 'PA1_Port1RewardDelay': array([8.6877, 8.7877]),
 'PA1_Port1Reward': array([8.7877, 8.8685]),
 'CloseValves': array([8.8685, 8.8686]),
 'Drinking': array([8.8686, 8.9686]),
 'Port1RewardDelay': array([nan, nan]),
 'Port2RewardDelay': array([nan, nan]),
 'Port3RewardDelay': array([nan, nan]),
 'Port4RewardDelay': array([nan, nan]),
 'PA1_Port2RewardDelay': array([nan, nan]),
 'PA1_Port3RewardDelay': array([nan, nan]),
 'PA1_Port4RewardDelay': array([nan, nan]),
 'Port1Reward': array([nan, nan]),
 'Port2Reward': array([nan, nan]),
 'Port3Reward': array([nan, nan]),
 'Port4Reward': array([nan, nan]),
 'PA1_Port2Reward': array([nan, nan]),
 'PA1_Port3Reward': array([nan, nan]),
 'PA1_Port4Reward': array([nan, nan]),
 'Punish': array([nan, nan]),
 'EarlyWithdrawal': array([nan, nan])}

In [70]:
bpod_object.trial_data[0]['States']

{'WaitForPosTriggerSoftCode': array([  0.  , 117.23]),
 'CueDelay': array([117.23, 117.24]),
 'WaitForResponse': array([117.24  , 120.0423]),
 'Port4RewardDelay': array([120.0423, 120.1423]),
 'Port4Reward': array([120.1423, 120.1976]),
 'CloseValves': array([120.1976, 120.1977]),
 'Drinking': array([120.1977, 120.2977]),
 'Port1RewardDelay': array([nan, nan]),
 'Port2RewardDelay': array([nan, nan]),
 'Port3RewardDelay': array([nan, nan]),
 'PA1_Port1RewardDelay': array([nan, nan]),
 'PA1_Port2RewardDelay': array([nan, nan]),
 'PA1_Port3RewardDelay': array([nan, nan]),
 'PA1_Port4RewardDelay': array([nan, nan]),
 'Port1Reward': array([nan, nan]),
 'Port2Reward': array([nan, nan]),
 'Port3Reward': array([nan, nan]),
 'PA1_Port1Reward': array([nan, nan]),
 'PA1_Port2Reward': array([nan, nan]),
 'PA1_Port3Reward': array([nan, nan]),
 'PA1_Port4Reward': array([nan, nan]),
 'Punish': array([nan, nan]),
 'EarlyWithdrawal': array([nan, nan])}

In [87]:
trial = 3
bpod_object.trial_data[trial]['States']['WaitForResponse'][1] - bpod_object.trial_data[trial]['States']['WaitForResponse'][0]

20.2621

In [77]:
# time between bpod first visual stimulus trigger and AUX master trigger onset in BPOD frame
bpod_object.trial_data[trial]['States']['CueDelay'][0] - bpod_object.trial_data[trial]['Events']['BNC1High'][0]

KeyError: 'BNC1High'

In [89]:
# time between bpod first visual stimulus trigger and AUX master trigger onset in AUX frame
bpod_trial_start = (event.Event()  &  "event_type='aux_bpod_visual'" &  scan_key ).fetch('event_start_time')
bpod_reward_start = (event.Event()  &  "event_type='aux_bpod_reward'" &  scan_key ).fetch('event_start_time')
auxgatetimestamp_start = (event.Event()  &  "event_type='main_track_gate'" &  scan_key ).fetch('event_start_time')

bpod_reward_start[trial] - bpod_trial_start[trial] -.01
# bpod_trial_start[0] - auxgatetimestamp_start[0] 

20.262200000000004

In [ ]:
bpod_object.trial_data

# Add Harp recording

In [ ]:
from adamacs.pipeline import behavior, event, scan
from adamacs.ingest.harp import HarpLoader, HarpLoader_sync
event_recording = event.BehaviorRecording.fetch('KEY')[0]
behavior.HarpRecording()

In [ ]:
event.BehaviorRecording() * session.SessionUser() * subject.User()

In [ ]:
event_recording

In [ ]:
# event.BehaviorRecording().delete

In [ ]:
scansi = "scan9FJ1ISOK"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*harp*bin"))

In [ ]:
scansi = "scan9FJ1ISOK"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*IMU_harp*csv"))

In [ ]:
harp_paths

In [ ]:
allchans = HarpLoader(harp_paths[0]).data_for_insert()

In [ ]:
allchans

In [ ]:
scansi = "scan9FJ1ISOK"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*2Pframes_harp*csv"))

In [ ]:
harp_paths

In [ ]:
allchans_sync = HarpLoader_sync(harp_paths[0]).data_for_insert()

In [ ]:
allchans_sync + allchans

# populate IMU data

In [ ]:
behavior.HarpRecording.populate()

In [ ]:
scansi = "scan9FJ842C3"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

In [ ]:
behavior.HarpRecording.Channel() & scan_key

In [ ]:
accelerometer = (behavior.HarpRecording.Channel() & scan_key & "channel_name LIKE 'IMU accelerometer %'").fetch("data")
gyroscope = (behavior.HarpRecording.Channel() & scan_key & "channel_name LIKE 'IMU gyroscope %'").fetch("data")
magnetometer = (behavior.HarpRecording.Channel() & scan_key & "channel_name LIKE 'IMU magnetometer %'").fetch("data")
twopframes = (behavior.HarpRecording.Channel() & scan_key & "channel_name LIKE '2p %'").fetch("data")
twopptime = (behavior.HarpRecording.Channel() & scan_key & "channel_name LIKE '2p %'").fetch("time")

In [ ]:
import matplotlib.pyplot as plt 

fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(15, 8))
# plt.rcParams['agg.path.chunksize'] = 10000  # Add this line if it does not rende


for i, arr in enumerate(accelerometer):
    axes[0].plot(arr, label=f'accelerometer {i+1}')
    
axes[0].set_ylim([-10000, 10000])
axes[0].set_ylabel("accelerometer")
axes[0].set_xlabel("Time [ms]")
axes[0].legend()

for i, arr in enumerate(gyroscope):
    axes[1].plot(arr, label=f'gyroscope {i+1}')
    
axes[1].set_ylabel("gyroscope")
axes[1].set_xlabel("Time [ms]")
axes[1].legend()

for i, arr in enumerate(magnetometer):
    axes[2].plot(arr, label=f'magnetometer {i+1}')
    
axes[0].set_ylim([-10000, 10000])
axes[2].set_ylabel("magnetometer")
axes[2].set_xlabel("Time [ms]")
axes[2].legend()


for i, arr in enumerate(twopframes):
    axes[3].plot(twopframes[0][:-1], label=f'2p frames {i+1}')
    
# axes[2].set_ylim([-10, 370])
axes[3].set_ylabel("2p frame")
axes[3].set_xlabel("Time [ms]")
axes[3].legend()

fig.suptitle(scan_key["scan_id"], fontsize=16)

plt.show() 

In [ ]:
event.EventType()

In [ ]:
auxgatetimestamp_start = (event.Event()  &  "event_type = 'HARP_gate'" &  scan_key ).fetch('event_start_time')
auxgatetimestamp_end = (event.Event()  &  "event_type = 'HARP_gate'" &  scan_key ).fetch('event_end_time')

print(auxgatetimestamp_start)
print(auxgatetimestamp_end)

In [ ]:
twopptimeoff = twopptime[0] + auxgatetimestamp_start * 1000
print(twopptimeoff[0] / 1000)
print(twopptimeoff[-1] / 1000)

In [ ]:
auxgatetimestamp_start